## Do the Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt




## Load the csv files

In [39]:
df_categories = pd.read_csv('categories.csv')
df_items = pd.read_csv('items.csv')
df_sales_train = pd.read_csv('sales_train.csv')
df_shops = pd.read_csv('shops.csv')
df_test = pd.read_csv('test.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')

In [40]:
df_sales_train.head()

#df_sales_train[df_sales_train['item_cnt_day'] == -1.0] 

#drop item_cnt_day == -1.0 cause number of items can't be negative
df_sales_train = df_sales_train[df_sales_train.item_cnt_day != -1.0]

#extract only the data which is from october 
df_sales_train_oct = df_sales_train.drop(df_sales_train.loc[df_sales_train['date_block_num'] != 33 ].index)

df_sales_train_oct

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [44]:
df_sales_train_oct = 

KeyError: 'shop_id'